In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import scipy.stats as ss

from collections import Counter
import math

# Работа с данными (к лаб. 1)

Загружаем датасет

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
n_train = train_df.shape[0]
n_test = test_df.shape[0]
print(f'{n_train=}')
print(f'{n_test=}')
df = train_df.append(test_df, ignore_index=True)
df.head()

Сразу отбрасываем ненужные столбцы с id записи потому что они не несут никакой полезной информации для определения будет ли удовлетворен клиент или нет

In [ ]:
df = df.drop(['Unnamed: 0', 'id'], axis='columns')

In [ ]:
df.info()

В датасете представлено 5 категориальных признаков, и 18 численных

In [ ]:
df.describe().T

Смотрим есть ли колонки с уникальными значениями

In [ ]:
print(f"unique value columns: {[col for col in df.columns if df[col].nunique() <= 1]}")

Смотрим есть ли дубликаты в записях

In [ ]:
df.duplicated().sum()

Для каждой из колонок смотрим сколько в ней потеряных значений (NaN'ов)

In [ ]:
# функция вернет датафрейм записями по каждму полю (общее число NaN'ов, процент NaN'ов от общего числа записей), для полей, у в которых есть хотя бы один NaN  
def missing_stats(df):
    missing = df.isna().sum().sort_values(ascending=False)
    missing = pd.concat([missing, missing / len(df) * 100], axis=1, keys=['Missing values', 'Missing percent'])
    return missing[missing['Missing values'] > 0.0]

missing_stats(df)

Пытаемся понять чем заменить пропущенные значения в колонке "Arrival Delay in Minutes". Для этого подсчитываем полетов без задержки прибытия

In [ ]:
non_zero_arrival_delay_flights_count = (df["Arrival Delay in Minutes"] > 0).sum()
print(f'Non-zero arrival delay flights percent: {non_zero_arrival_delay_flights_count / len(df * 100)}')

Видно, что подавляющее число полетов проходит без задержки прибытия, значит смело заменяем пропущенные значения нулем. Хотя, так как данных довольно много, можно было бы и отбросить "бракованные" записи

In [ ]:
mode = df['Arrival Delay in Minutes'].mode()[0]
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(mode)

# проверяем что все пропущенные значения теперь заполнены
missing_stats(df)

## Исследование данных

### Исследование целевой переменной

In [ ]:
sns.countplot(x='satisfaction', data=df)

Видно, что недовольных или равнодушных клиентов больше, чем удовлетворенных. Однако дисбаланс классов небольшой.

### Числовые признаки

In [ ]:
df_numerical = df.select_dtypes([np.number])
df_numerical = pd.concat([df['satisfaction'].map({'neutral or dissatisfied' : 0, 'satisfied' : 1}), df_numerical], axis=1)
df_numerical

In [ ]:
df_numerical.hist(figsize=(15,20))
plt.figure()

- Возраст пассажииров, по всей видимости, имеет бимодальное распределение с пиками в районе 25 и 40 лет. Однако пасажиры возрасте от 40 до 60 лет преобладают.

- Распределение расстояния перелета похоже на экспонециально, но нужно уточнять.

- Задержки вылета и прибытия, похоже, в абсолютном большинстве случаев близки к нулю.

- Величины, обозначающие уровень удовлетворенности клиента той или иной отдельной характеристикой имеют сходства. Почти у всех из них очень мало нулевых оценок, и самая распространенная оценка 4 балла. Эти величины, наверняка, нормально распределены с перекосом в большую сторону. Нужно уточнять.

### Зависимость целевой переменной от числовых признаков

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Age').add_legend()

Видно, что распределения по возрастам довольно сильно перекрываюся. Однако среди пассажиров с возрастом в диапазоне 8-40 больше недовольных, в диапазоне 40-60 довольных почти в 2 раза больше, чем недовольных, а в диапазоне 60-80 гораздо больше неудовлетворенных.

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Flight Distance').add_legend()

Распределения сильно перекрываюся. Видно, что среди пассажиров, совершающих перелет на дистанцию примерно до 1200 км, но на дистанциях 1200-4000 км число довольных более чем вдвое превышает число недовольных

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Inflight wifi service').add_legend()

- Как ни странно, если оценка wi-fi'я не применима, видимо ввиду его отсутствия ?? (в таблице оценка = 0), то пассажир почти гарантировано останется доволен полетом
- Пассажиры, поставившие 1, 2, или 3 гораздо чаще остаются недовольны и всем полетом в целом
- Среди пассажиров, поставивших 4 за эту характеристику совсем немного больше удовлетворенных полетом в целом
- Если пассажир поставил оценку 5 за эту характеристику, то он почти гарантированно удовлетворен полетом

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Departure/Arrival time convenient').add_legend()

Отдельно эта оценка почти не влияет на удовлетворенность клиента в целом, т.к. число довольных и недовольных почти равно для любого балла, разве что для 4-х неудовлетворенных слегка больше. Не очень информативный признак, вероятно можно отбросить ??

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Ease of Online booking').add_legend()

- Если оценка не применима, то число довольных пассажиров в 2 раза превышает число недовольных
- Среди пассажиров, поставивших 1, немного больше недовольных
- Пассажиры, поставивщие оценки 2, 3 в два раза чаще недовольны
- Среди пассажиров, поставивших 4, немного больше довольных
- Среди пассажиров поставивших 5 гораздо больше оставшихся довольными

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Gate location').add_legend()

- Число довольных и недовольных пассажиров, поставивших 0-2 примерно равно
- Среди поставивших 3 или 4 заметно больше недовольных
- Среди поставивших 5 больше довольных

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Food and drink').add_legend()

- Пассажир поставивший 1 с большой вероятностью останется недоволен полетом
- Поставившие 2-3 скорее недовольны, чем довольны
- Среди поставивших 4 или 5 заметно больше довольных

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Online boarding').add_legend()

- Если пассажир поставил оценку 1-2 за эту характеристуку, то он большой вероятностью останется недоволен
- Пассажиры, поставившие 4, в 2 два раза чаще довольны
- Пассажир, поставивший 5, с огромной вероятностью останется доволен полетом

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Seat comfort').add_legend()

- Пассажиры, поставившие 1-3 за эту характеристику в 2-х случаях из 3-х останутся недовольными
- Напротив, пассажиры, поставившие 4-5 более, чем в 2 раза чаще оказываются довольны полетом

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Inflight entertainment').add_legend()

- Пассажиры, поставившие 1-3 за эту характеристику примерно в 2 раза чаще останутся недовольными
- Пассажиры, поставившие 4-5 более, чем в 2 раза чаще оказываются довольны полетом

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'On-board service').add_legend()

- Пассажиры, поставившие 1-3 за эту характеристику примерно в 2 раза чаще останутся недовольными
- Пассажиры, поставившие 4-5 более, чем в 2 раза чаще оказываются довольны полетом

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Leg room service').add_legend()

- Пассажиры, поставившие 1-3 за эту характеристику примерно в 2 раза чаще останутся недовольными
- Пассажиры, поставившие 4-5 более, чем в 2 раза чаще оказываются довольны полетом

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Baggage handling').add_legend()

- Пассажиры, поставившие 1-3 за эту характеристику примерно в 2 раза чаще останутся недовольными
- Пассажиры, поставившие 4 более, немного чаще оказываются довольны полетом
- Пассажиры, поставившие 5 более, чем в 2 раза чаще оказываются довольны полетом

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Checkin service').add_legend()

- Пассажиры, поставившие 1-2 за эту характеристику более чем в 2 раза чаще останутся недовольными
- Пассажиры, поставившие 5 более, чем в 2 раза чаще оказываются довольны полетом

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Inflight service').add_legend()

- Пассажиры, поставившие 1-3 чаще остаются недовольными
- Пассажиры, поставившие 4 или 5 более, чаще оказываются довольны

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7).map(sns.distplot, 'Cleanliness').add_legend()

- Пассажиры, поставившие 1-3 чаще остаются недовольными
- Пассажиры, поставившие 5 более, чаще оказываются довольны

In [ ]:
sns.FacetGrid(df, hue='satisfaction', height=7, xlim=(0, 100)).map(sns.distplot, 'Arrival Delay in Minutes', bins=200).add_legend()
sns.FacetGrid(df, hue='satisfaction', height=7, xlim=(0, 100)).map(sns.distplot, 'Departure Delay in Minutes', bins=200).add_legend()

На графике задержки прибытия видно, что уже во втором и всех последующих бинах гистограммы значительно больше недовольных.
На графике задержки отправки ситуация несколько мягче: в первых двух бинах большее довольных, а в последующих не такая большая разница между довольными и недовольными. Т.е. пассажиры, терпимее относятся к задержке отправки, чем к задержке прибытия, что вполне логчно. Значит задержка отправки для нас менее информативна, если отбрасывать из двух.

Также рискну предположить что "переход" в районе 1-го бакета гистограммы с удвлетвоенного на не удовлетворенного происхоит из-за самого факта задержки.
Для проверки того построим гистограммы распределения удовлеторенных и неудовлетворенных пассажиров для данных с задержкой и без.

In [ ]:
zero_departure_delay = df[df['Departure Delay in Minutes'] == 0]
non_zero_departure_delay = df[df['Departure Delay in Minutes'] > 0]
zero_arrival_delay = df[df['Arrival Delay in Minutes'] == 0]
non_zero_arrival_delay = df[df['Arrival Delay in Minutes'] > 0]

fig, ax = plt.subplots(2, 2, sharey=True, figsize=(8, 8))
fig.suptitle('Satisfaction with/without delays')
fig.subplots_adjust(hspace = 0.3)
sns.histplot(x='satisfaction', data=zero_departure_delay, ax=ax[0, 0])
ax[0,0].set_title('Zero departure delay')
sns.histplot(x='satisfaction', data=non_zero_departure_delay, ax=ax[0, 1])
ax[0,1].set_title('Non-zero departure delay')
sns.histplot(x='satisfaction', data=zero_arrival_delay, ax=ax[1, 0])
ax[1,0].set_title('Zero arrival delay')
sns.histplot(x='satisfaction', data=non_zero_arrival_delay, ax=ax[1, 1])
ax[1,1].set_title('Non-zero arrival delay')

- Видно, что среди рейсов без задержки отправки/прибытия удовлетворенных клиентов больше, чем среди рейсов с задержкой. 
- Кроме того распределения для задерки отправки и задержки прибытия почти одинаковы, что лишний раз свидетельствует о том, что одну из переменных можно отбросить.

In [ ]:
numerical_cols = list(df_numerical.dtypes[df_numerical.dtypes != 'object'].index)

numerical_cols.remove('satisfaction')
numerical_cols.insert(0, 'satisfaction')

pp = sns.pairplot(df_numerical[numerical_cols], height=4)
pp.fig.savefig('pairplot.png')

- Видна явная линейная зависимость видна между задержкой вылета 'Departure Delay in Minutes' и задержкой прибытия 'Arrival Delay in Minutes'. Большее зависимостй не наблюдается.
- Так же видны выбросы для задержкек вылета прибытия

Cоберем статистику по количеству полетов с задержкой больше каждой величины кратной 100 со 100 до 1600

In [ ]:
stats = []
for i in range(1, 17):
    min_delay = i * 100
    count = (df['Arrival Delay in Minutes'] > i * 100).sum()
    stats.append([min_delay, count, count / len(df) * 100])
pd.DataFrame(stats, columns=['Min delay', 'Count', 'Percent'])

Видно, что рейсы с задержками прибытия более чем на 400 минут составляют порядка менее десятой доли процента. Отбросим все что больше или равно 400 ?? считая выбросами.

In [ ]:
df = df[df['Arrival Delay in Minutes'] <= 400]
df = df[df['Departure Delay in Minutes'] <= 400]

Расчет корреляционной матрицы

In [ ]:
corr_mat = df_numerical.corr()
plt.subplots(figsize=(15,9))
sns.heatmap(corr_mat[np.abs(corr_mat) >= 0.4], 
            cmap='RdBu_r',
            vmax=1.0, vmin=-1.0,
            linewidth=0.1,
            annot=True,
            annot_kws={"size":8})

In [ ]:
row = corr_mat.iloc[0].abs()
row = row.sort_values(axis='index', ascending=False)
row[row > 0.3]

Рассморим подробее корреляцию целевой перемнной от входных числовых. Видно, что целевая переменная 'satisfaction' имеет среднюю корреляцию с единственной независимой переменной 'Online-boarding', а также слабо коррелирована с еще 5-ю еременными: 'Inflight entertainment', 'Seat comfort', 'On-board service', 'Leg room service', 'Cleanliness'

### Категориальные признаки

Построим распределение категориальных признаков

In [ ]:
categorical_cols = list(df.dtypes[df.dtypes == 'object'].index) 
categorical_cols.remove('satisfaction')

fig, ax = plt.subplots(2, 2, figsize=(10, 10), sharey=True)
for i, name in enumerate(categorical_cols):
    df_plot = df.groupby(['satisfaction', name]).size().reset_index().pivot(columns='satisfaction', index=name, values=0)
    p = df_plot.plot(kind='bar', stacked=True, ax=ax[i//2, i%2])
    p.set_xticklabels(p.get_xticklabels(), rotation = 0)

- Пассажиров мужского и женского рода примерно одинаковое число, кроме того, в обоих классах распределение довольных/недовольных примерно 45/55, похоже это не очень информативный признак
- Лояльных клиентов значительно больше, чем нелояльных, среди лояльных половина довольны, остальные - нет, среди нелояльных значительно больше недовольных
- Гораздо чаще совершаются деловые поездки, чем индивидуальные перелеты. Среди пассаижиров, совершающих деловую поездку немного больше довольых.
Пассажиры, путешествущие "для себя" в абсолютном большинстве случаев оказываются недовольны перелетом
- Пассажиров, летящих бизнес классом чуть-чуть больше, чем летящих в эконом классе. В клсс эко-плюс летает очень мало пассажиров. Причем довольных в бизнес-классе примерно в 3 раза больше. В эко и эко-плюс ситуация прямо противоположная -- там абсоютное большинство пассжиров недовоьны перелетом

Рассморим теперь корреляцию целевой перемнной от входных категориалных

In [ ]:
df_categorical = df.select_dtypes([object])
df_categorical = df_categorical[['satisfaction', 'Gender', 'Customer Type', 'Type of Travel', 'Class']]

Для категориальных переменных сущестут аналог корреляции -- Cramér's V,
а так же коэффицент неопределенности Theil's U, показывающий насколько хорошо мы можем предсказать одну переменную, учитывая другую.

- [Суть](https://towardsdatascience.com/the-search-for-categorical-correlation-a1cf7f1888c9)
- [Реализация Cramér's V](https://stackoverflow.com/questions/46498455/categorical-features-correlation/46498792#46498792)
- [Реализация Theil's U](https://stackoverflow.com/questions/54931514/theils-u-1-theils-u-2-forecast-coefficient-formula-in-python)

In [ ]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

def conditional_entropy(x,y):
    y_counter = Counter(y)
    xy_counter = Counter(list(zip(x,y)))
    total_occurrences = sum(y_counter.values())
    entropy = 0
    for xy in xy_counter.keys():
        p_xy = xy_counter[xy] / total_occurrences
        p_y = y_counter[xy[1]] / total_occurrences
        entropy += p_xy * math.log(p_y/p_xy)
    return entropy

def theils_u(x, y):
    s_xy = conditional_entropy(x,y)
    x_counter = Counter(x)
    total_occurrences = sum(x_counter.values())
    p_x = list(map(lambda n: n/total_occurrences, x_counter.values()))
    s_x = ss.entropy(p_x)
    if s_x == 0:
        return 1
    else:
        return (s_x - s_xy) / s_x

def apply(df, f):
    n = df.shape[1]
    mat = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            mat[i][j] = f(df[df.columns[i]], df[df.columns[j]])
    ndf = pd.DataFrame(mat, index=df.columns, columns=df.columns)
    return ndf

Расчитаем "корреляционную" матрицу для категориальных признаков

In [ ]:
cramers_v_mat = apply(df_categorical[:df.shape[0]], cramers_v)

corr_mat = df_numerical.corr()
plt.subplots(figsize=(9,5))
sns.heatmap(cramers_v_mat[(cramers_v_mat >= 0.4) | (cramers_v_mat < -0.4)], 
            cmap='RdBu_r',
            vmax=1.0, vmin=-1.0,
            linewidth=0.1,
            annot=True,
            annot_kws={"size":8})

In [ ]:
row = cramers_v_mat.iloc[0]
row.sort_values(axis='index', ascending=False).head()

Рассморим подробее корреляцию целевой перемнной от входных категориальных. Видно, что целевая переменная 'satisfaction' имеет среднюю корреляцию с единственной независимой переменной 'Class' и 'Type of Travel', а также довольно слабо коррелирована переменной с 'Customer Type'

Расчитаем коэффиценты неопределенности

In [ ]:
theils_u_mat = apply(df_categorical, theils_u)

f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(theils_u_mat, square=True, annot=True, vmax=1.0, vmin=-1.0, cmap='RdBu_r', annot_kws={"size":8})

По исследованию коэффициентов неопределенности нельзя отметить сильного влияния категориальных переменных друг на друга.
Итого, судя по всему, все категориальные переменные достаточно (линейно) независимы друг от друга.

- Возможно стоит сгенерировать такие признаки как "общее удовлетворение сервисом на борту", "общее удовлетворение сервисом на онлайн" и "общее удовлетворение сервисом в аэропорту" как среднее касающихся этих признаков ценок.
- Также возможно стот добавить признак-флаг, говорящий была ли хоть какая-нибудь задержка прибытия

In [ ]:
df['Overall Satisfaction on board'] = 0.125 * (df['Inflight wifi service'] + df['Food and drink'] + df['Seat comfort'] + df['Inflight entertainment'] + df['On-board service'] + df['Leg room service'] +  df['Inflight service'] +  df['Cleanliness'])
df['Overall Satisfaction online'] = 0.5 * (df['Online boarding'] + df['Ease of Online booking'])
df['Overall Satisfaction airport'] = 1/3 * (df['Gate location'] + df['Baggage handling'] + df['Checkin service'])

df['Arrival delayed'] = (df['Arrival Delay in Minutes'] > 0).astype('int')

Т.о. наиболее информатиыными признаками будут:
- 'Class', 'Type of Travel' -- категориальные;
- 'Online boarding', 'Inflight entertainment', 'Seat comfort', 'On-board service', 'Leg room service', 'Cleanliness', 'Arrival delayed' -- числовые

Пока так, дальше методом проб и ошибок по резульатам обучения моделей

## Преобразование данных

Проверим имеют ли числовые переменные перекос

In [ ]:
numeric_feats = df.dtypes[df.dtypes != "object"].index
skewed_feats = df[numeric_feats].apply(lambda x: ss.skew(x.dropna())).sort_values(ascending=False)
print("\nПерекос в численных признаках: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness

Несколько признаков имеют значительный перекос, скорректируем его с помощью преобразования Бокса-Кокса

In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    df[feat] = boxcox1p(df[feat], lam)
df

Также перед обучением моделей необходимо нормализовать числовые данные, а категориальные переменные заменить на бинарные признаки при помощи One hot encoding

In [ ]:
from sklearn import preprocessing

numerical_col = ['Online boarding', 'Inflight entertainment', 'Seat comfort', 'On-board service', 'Leg room service', 'Cleanliness', 'Arrival delayed']
categorical_col = ['Class', 'Type of Travel']

scaler = preprocessing.StandardScaler().fit(df[numerical_col])

t = scaler.transform(df[numerical_col])

y = df['satisfaction'].map({'neutral or dissatisfied' : 0, 'satisfied' : 1})
categorical_X = pd.get_dummies(df[categorical_col])
numerical_X = pd.DataFrame(t, columns=numerical_col, index=categorical_X.index)

X = pd.concat([categorical_X, numerical_X], axis=1)

train_slice = slice(0, n_train)
test_slice = slice(n_train, n_train + n_test)

x_train = X[train_slice].reset_index()
y_train = y[train_slice].reset_index()

x_test = X[test_slice].reset_index()
y_test = y[test_slice].reset_index()